## Get AWS terraform documentation from official site

In [ ]:
import requests
import json

# Get provider-docs definition through API call
url = f"https://registry.terraform.io/v2/provider-versions/67250?include=provider-docs"

headers = {
    "Content-Type": "application/json",
    "User-Agent": "your-user-agent"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json() # Prints provider details
else:
    print(f"Error: {response.status_code}, {response.text}")

# Extract the command and resource id from the response
link_extentions = [x['links']['self'] for x in data['included']]
command_prompts = ['Create Terraform with ' + ' '.join(x['attributes']['title'].split("_")) for x in data['included']]

# Get code sample from links
code_urls = ["https://registry.terraform.io/" + ex for ex in link_extentions]

# Get Code Sample from each link
terraform_code = []

for url in code_urls:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json() # Prints provider details
    else:
        print(f"Error: {response.status_code}, {response.text}")
        
    terraform_code.append(data['data']['attributes']['content'])

## Save into jsonl file

In [ ]:
outputs_data = []
for prompt, completion in zip(command_prompts, terraform_code):
    outputs_data.append({
        "prompt": prompt,
        "completion": completion
    })

# Save as JSONL
with open("terraform_aws_docs_training_data2.jsonl", "w") as f:
    for entry in outputs_data:
        f.write(json.dumps(entry) + "\n")


In [ ]:
print(f"Generated {len(outputs_data)} prompt-completion pairs.")
outputs_data[0]